In [1]:
as = [3.3 , 4.3] #prices public info
cs = [3.0,4.0] #costs, private info
n= 2
lag = 1
#info set is lagged 1 period back,

function init_Ws(as,cs,n,lag, W0)
    Ws = Dict()
    argumento=fill(as, n*lag + 1 )
    cartesian_as=collect(Iterators.product(argumento...))
    
    argumento= fill(cs, lag + 1 )
    cartesian_cs = collect(Iterators.product(argumento...))
    for c=cartesian_cs
        for a=cartesian_as
                Ws["$(a),$(c)"] = (rand(W0),0)
        end
    end
    return Ws
end

W0 = init_Ws(as,cs,n,lag,[10,11,12])

Dict{Any,Any} with 32 entries:
  "(4.3, 3.3, 3.3),(3.0, 4.0)" => (10, 0)
  "(3.3, 3.3, 4.3),(3.0, 3.0)" => (10, 0)
  "(4.3, 3.3, 3.3),(4.0, 3.0)" => (11, 0)
  "(3.3, 4.3, 4.3),(4.0, 4.0)" => (11, 0)
  "(4.3, 3.3, 4.3),(4.0, 4.0)" => (12, 0)
  "(4.3, 3.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(4.0, 4.0)" => (11, 0)
  "(4.3, 3.3, 4.3),(3.0, 4.0)" => (11, 0)
  "(4.3, 4.3, 4.3),(4.0, 4.0)" => (10, 0)
  "(3.3, 3.3, 4.3),(4.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(3.0, 4.0)" => (12, 0)
  "(3.3, 4.3, 3.3),(4.0, 4.0)" => (12, 0)
  "(4.3, 4.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(4.3, 4.3, 3.3),(3.0, 3.0)" => (12, 0)
  "(3.3, 3.3, 3.3),(4.0, 3.0)" => (12, 0)
  "(3.3, 4.3, 4.3),(3.0, 3.0)" => (12, 0)
  "(4.3, 3.3, 3.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 3.3, 3.3),(3.0, 3.0)" => (12, 0)
  "(4.3, 4.3, 4.3),(4.0, 3.0)" => (12, 0)
  "(4.3, 4.3, 3.3),(4.0, 4.0)" => (12, 0)
  "(3.3, 4.3, 4.3),(3.0, 4.0)" => (10, 0)
  "(3.3, 4.3, 3.3),(3.0, 3.0)" => (10, 0)
  "(4.3, 4.3, 3.3),(4.0, 3.0)" => (12, 0)
  "

In [2]:
function update_m(as,cs,n,lag,Ws)
    
    argumento=fill(as, n*lag)
    cartesian_as=collect(Iterators.product(argumento...))
    argumento= fill(cs, lag + 1 )
    cartesian_cs = collect(Iterators.product(argumento...))
    
    ms = Dict()
    for other_a=cartesian_as
        for c=cartesian_cs
            for my_a=as
                #println("my_a ", my_a)
                #check if key is there initilize to my_a
                if ! haskey(ms, "$(other_a),$(c)") 
                    ms["$(other_a),$(c)"] = my_a
                end
                
                current_pol = ms["$(other_a),$(c)"]
                current_state = Tuple(append!([current_pol],other_a ))
                other_state = Tuple(append!([my_a],other_a))
                
                #if already there set to my a if greater
                #println(my_a, ", ", current_state,", " ,other_state, ", ", Ws["$(current_state),$(c)"], ", ",  Ws["$(other_state),$(c)"] )
                if Ws["$(current_state),$(c)"][1] <= Ws["$(other_state),$(c)"][1]
                    ms["$(other_a),$(c)"] = my_a
                end
            end
        end
    end
    m(as,omegas) = ms["$(Tuple(as)),$(Tuple(omegas))"]
    return m
end


m0 = update_m(as,cs,n,lag,W0)
print(m0)
m0((4.3, 4.3),(3.,3.))

getfield(Main, Symbol("#m#3")){Dict{Any,Any}}(Dict{Any,Any}("(3.3, 3.3),(3.0, 4.0)"=>3.3,"(4.3, 4.3),(4.0, 4.0)"=>3.3,"(3.3, 4.3),(3.0, 3.0)"=>4.3,"(4.3, 4.3),(3.0, 3.0)"=>3.3,"(3.3, 3.3),(3.0, 3.0)"=>4.3,"(3.3, 3.3),(4.0, 3.0)"=>3.3,"(4.3, 4.3),(3.0, 4.0)"=>4.3,"(4.3, 3.3),(4.0, 3.0)"=>4.3,"(3.3, 4.3),(3.0, 4.0)"=>3.3,"(3.3, 4.3),(4.0, 3.0)"=>4.3,"(3.3, 4.3),(4.0, 4.0)"=>4.3,"(3.3, 3.3),(4.0, 4.0)"=>3.3,"(4.3, 4.3),(4.0, 3.0)"=>4.3,"(4.3, 3.3),(3.0, 4.0)"=>4.3,"(4.3, 3.3),(4.0, 4.0)"=>4.3,"(4.3, 3.3),(3.0, 3.0)"=>4.3))

3.3

In [3]:
#reset costs every 2 periods
function update_private_info(m,current_as,current_cs,cs,n,lag,t)
    next_cs = copy(current_cs)
    
    #update costs
    for i=1:n
        next_ci = collect(next_cs[i])
        next_ci[1] =current_cs[i][1]
        if mod(t,2) == 0
            next_ci[1] = cs[rand(1:end)]
        end

        for s=0:(lag-1)
            next_ci[end-s] = current_cs[i][end-s-1]
        end
        next_cs[i] = Tuple(next_ci)
    end
    return next_cs
end


function update_public_info(m,current_as,current_cs,cs,n,lag,t)
    next_as = copy(current_as)
    for i=1:n
        next_asi = collect(next_as[i])
        for j=0:(n-1)
            next_asi[mod(j+i-1,n)+1] = m(current_as[i],current_cs[i])
        end
        next_as[i] = Tuple(next_asi)
    end
    return next_as
end


function update_info(m,current_as,current_cs,cs,n,lag,t)
    next_cs = update_private_info(m,current_as,current_cs,cs,n,lag,t)
    next_as = update_public_info(m,current_as,current_cs,cs,n,lag,t)
    return next_as, next_cs, t+1
end

update_info(m0, [(4.3, 4.3),(4.3,4.3)],[(4.,4.),(4.,4.)],cs,n,lag,2)

(Tuple{Float64,Float64}[(3.3, 3.3), (3.3, 3.3)], Tuple{Float64,Float64}[(3.0, 4.0), (3.0, 4.0)], 3)

In [4]:
BETA = .9
W0 = init_Ws(as,cs,n,lag,[10,11,12])

function update_ws(m,current_as,current_cs, n, beta, Ws)
    

    profit = zeros(n)
    policies = zeros(n)
    
    for i=1:n
        policies[i] = m( current_as[i], current_cs[i] )
    end
    for i=1:n
        share = 1 - exp(policies[i]) / ( 1+sum(exp.(policies) ))
        state = Tuple( append!( [policies[i]],current_as[i] ) )
        profit[i] = share*(policies[i] - current_cs[i][1]) + beta*Ws["$(state),$(current_cs[i])"][1]
    end
    
    for i=1:n
        state = Tuple( append!( [policies[i]],current_as[i] ) )
        weight = 1/( Ws["$(state),$(current_cs[i])"][2] +1 )
        new_w = weight*profit[i] + (1-weight)*Ws["$(state),$(current_cs[i])"][1]
        new_counter = Ws["$(state),$(current_cs[i])"][2] +1
        Ws["$(state),$(current_cs[i])"]= (new_w,new_counter)
    end
    
    return Ws
end

competition(x,y) = 3.3

update_ws( m0,[(4.3, 4.3),(4.3,4.3)],[(3.,3.),(3.,3.)], n, 0, W0 )

Dict{Any,Any} with 32 entries:
  "(4.3, 3.3, 3.3),(3.0, 4.0)" => (10, 0)
  "(3.3, 3.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(4.3, 3.3, 3.3),(4.0, 3.0)" => (12, 0)
  "(3.3, 4.3, 4.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 3.3, 4.3),(4.0, 4.0)" => (12, 0)
  "(4.3, 3.3, 4.3),(3.0, 3.0)" => (12, 0)
  "(3.3, 3.3, 4.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 3.3, 4.3),(3.0, 4.0)" => (11, 0)
  "(4.3, 4.3, 4.3),(4.0, 4.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(4.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(3.0, 4.0)" => (10, 0)
  "(3.3, 4.3, 3.3),(4.0, 4.0)" => (11, 0)
  "(4.3, 4.3, 4.3),(3.0, 3.0)" => (12, 0)
  "(4.3, 4.3, 3.3),(3.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 3.3),(4.0, 3.0)" => (11, 0)
  "(3.3, 4.3, 4.3),(3.0, 3.0)" => (0.152716, 2)
  "(4.3, 3.3, 3.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 3.3, 3.3),(3.0, 3.0)" => (11, 0)
  "(4.3, 4.3, 4.3),(4.0, 3.0)" => (11, 0)
  "(4.3, 4.3, 3.3),(4.0, 4.0)" => (10, 0)
  "(3.3, 4.3, 4.3),(3.0, 4.0)" => (11, 0)
  "(3.3, 4.3, 3.3),(3.0, 3.0)" => (12, 0)
  "(4.3, 4.3, 3.3),(4.0, 3.0)" => (11, 

In [19]:
iters=10000
as = [2.3, 3.3, 4.3,5.3] #prices public info
cs = [3.,4.] #costs, private info
n= 2
lag = 1
Ws = init_Ws(as,cs,n,lag,[1.8,1.9,2.0])
ms = update_m(as,cs,n,lag,Ws)
as_current = [(3.3, 3.3),(3.3,3.3)]
cs_current = [(3.,3.),(3.,3.)]
t=0
beta = .9

for i=1:iters
    as_current, cs_current, t = update_info(ms,as_current,cs_current,cs,n,lag,t)
    Ws= update_ws(ms,as_current,cs_current, n, beta, Ws)
    ms= update_m(as,cs,n,lag,Ws)
end

for (k,v) in Ws
    if v[2] > 40
        println(k,v)
    end
end

for a=as
    tally = 0
    for (k,v) in Ws
        action = parse(Float64,k[2:4])
        if action ==a
            tally = tally + v[2]
        end
    end
    println("$(a): $(tally)")
end

(5.3, 5.3, 5.3),(4.0, 4.0)(3.8721931624943657, 5005)
(4.3, 4.3, 4.3),(4.0, 3.0)(1.9314627614188355, 365)
(5.3, 4.3, 4.3),(4.0, 4.0)(3.711889119582187, 2505)
(4.3, 3.3, 3.3),(4.0, 3.0)(1.8620736670680091, 1416)
(4.3, 5.3, 5.3),(4.0, 3.0)(1.8570360123958096, 723)
(5.3, 4.3, 4.3),(3.0, 3.0)(5.793026560347197, 1773)
(4.3, 3.3, 3.3),(3.0, 3.0)(5.035140501465632, 2138)
(3.3, 5.3, 5.3),(3.0, 3.0)(2.0743826929849947, 3555)
(5.3, 5.3, 5.3),(3.0, 4.0)(5.9977945078820065, 2508)
2.3: 2
3.3: 3559
4.3: 4643
5.3: 11796


In [20]:
as = [2.3, 3.3, 4.3,5.3] #prices public info
cs = [3.,4.] #costs, private info
n= 2
lag = 1
Ws = init_Ws(as,cs,n,lag,[1.8,1.9,2.0])
ms = update_m(as,cs,n,lag,Ws)
as_current = [(3.3, 3.3),(3.3,3.3)]
cs_current = [(3.,3.),(3.,3.)]
t=0
beta = .5

for i=1:iters
    as_current, cs_current, t = update_info(ms,as_current,cs_current,cs,n,lag,t)
    Ws= update_ws(ms,as_current,cs_current, n, beta, Ws)
    ms= update_m(as,cs,n,lag,Ws)
end

for (k,v) in Ws
    if v[2] > 40
        println(k,v)
    end
end

for a=as
    tally = 0
    for (k,v) in Ws
        action = parse(Float64,k[2:4])
        if action ==a
            tally = tally + v[2]
        end
    end
    println("$(a): $(tally)")
end

(5.3, 5.3, 5.3),(4.0, 4.0)(1.011142995459199, 7685)
(5.3, 4.3, 4.3),(4.0, 3.0)(1.0075727481584038, 2437)
(4.3, 4.3, 4.3),(3.0, 3.0)(1.603787381120259, 7385)
(4.3, 5.3, 5.3),(3.0, 4.0)(1.6257292376870918, 2436)
2.3: 12
3.3: 12
4.3: 9837
5.3: 10139


In [21]:
as = [2.3, 3.3, 4.3,5.3] #prices public info
cs = [2.,3.] #costs, private info
n= 2
lag = 1
Ws = init_Ws(as,cs,n,lag,[1.8,1.9,2.0])
ms = update_m(as,cs,n,lag,Ws)
as_current = [(3.3, 3.3),(3.3,3.3)]
cs_current = [(2.,2.),(2.,2.)]
t=0
beta = .9

for i=1:iters
    as_current, cs_current, t = update_info(ms,as_current,cs_current,cs,n,lag,t)
    Ws= update_ws(ms,as_current,cs_current, n, beta, Ws)
    ms= update_m(as,cs,n,lag,Ws)
end

for (k,v) in Ws
    if v[2] >= 1
        println(k,v)
    end
end

for a=as
    tally = 0
    for (k,v) in Ws
        action = parse(Float64,k[2:4])
        if action ==a
            tally = tally + v[2]
        end
    end
    println("$(a): $(tally)")
end



(2.3, 5.3, 5.3),(2.0, 3.0)(2.2498053888083693, 1513)
(2.3, 3.3, 3.3),(3.0, 3.0)(1.4332922520917557, 2)
(2.3, 4.3, 4.3),(3.0, 3.0)(1.1330410057616673, 1)
(2.3, 3.3, 3.3),(2.0, 2.0)(2.2970284619070838, 991)
(5.3, 2.3, 2.3),(3.0, 2.0)(4.5654389906907635, 494)
(5.3, 2.3, 2.3),(2.0, 2.0)(7.879340538452782, 2011)
(3.3, 4.3, 4.3),(3.0, 3.0)(1.898300732600596, 2)
(5.3, 2.3, 2.3),(3.0, 3.0)(1.919447736039961, 1)
(3.3, 5.3, 5.3),(3.0, 2.0)(1.9777486822413826, 2010)
(5.3, 5.3, 5.3),(2.0, 2.0)(8.303887584718208, 4482)
(5.3, 4.3, 4.3),(3.0, 3.0)(4.517929177242982, 1016)
(3.3, 2.3, 2.3),(3.0, 2.0)(1.8864407849630787, 1)
(4.3, 3.3, 3.3),(3.0, 3.0)(5.238280597067956, 2009)
(5.3, 5.3, 5.3),(3.0, 3.0)(5.954686326480337, 4476)
(3.3, 4.3, 4.3),(2.0, 3.0)(5.419262674024609, 991)
2.3: 2507
3.3: 3004
4.3: 2009
5.3: 12480


In [22]:
as = [2.3, 3.3, 4.3,5.3] #prices public info
cs = [2.,3.] #costs, private info
n= 2
lag = 1
Ws = init_Ws(as,cs,n,lag,[1.8,1.9,2.0])
ms = update_m(as,cs,n,lag,Ws)
as_current = [(3.3, 3.3),(3.3,3.3)]
cs_current = [(2.,2.),(2.,2.)]
t=0
beta = .5

for i=1:iters
    as_current, cs_current, t = update_info(ms,as_current,cs_current,cs,n,lag,t)
    Ws= update_ws(ms,as_current,cs_current, n, beta, Ws)
    ms= update_m(as,cs,n,lag,Ws)
end

for (k,v) in Ws
    if v[2] >= 1
        println(k,v)
    end
end

for a=as
    tally = 0
    for (k,v) in Ws
        action = parse(Float64,k[2:4])
        if action ==a
            tally = tally + v[2]
        end
    end
    println("$(a): $(tally)")
end

(4.3, 4.3, 4.3),(2.0, 3.0)(1.6350802927839132, 1)
(4.3, 2.3, 2.3),(3.0, 3.0)(1.6432147200265366, 2)
(2.3, 3.3, 3.3),(3.0, 3.0)(0.332456594522479, 1)
(4.3, 2.3, 2.3),(2.0, 3.0)(2.6836837597451186, 1)
(4.3, 3.3, 3.3),(3.0, 2.0)(1.55438006608638, 1)
(2.3, 4.3, 4.3),(3.0, 2.0)(0.38245659452247904, 1)
(2.3, 4.3, 4.3),(3.0, 3.0)(0.332456594522479, 1)
(4.3, 5.3, 5.3),(2.0, 3.0)(2.3107648451700475, 2481)
(3.3, 4.3, 4.3),(2.0, 2.0)(1.61176997247289, 2)
(2.3, 3.3, 3.3),(3.0, 2.0)(0.332456594522479, 1)
(5.3, 2.3, 2.3),(2.0, 3.0)(1.8962782259357214, 1)
(5.3, 2.3, 2.3),(2.0, 2.0)(2.654107812706762, 2)
(4.3, 4.3, 4.3),(2.0, 2.0)(2.648411367144788, 7534)
(3.3, 3.3, 3.3),(3.0, 3.0)(1.2643956607664226, 1)
(4.3, 4.3, 4.3),(3.0, 2.0)(1.3166835565104462, 2482)
(2.3, 5.3, 5.3),(3.0, 3.0)(0.2330410057616673, 1)
(3.3, 3.3, 3.3),(2.0, 2.0)(1.9450334541200396, 10)
(2.3, 4.3, 4.3),(2.0, 3.0)(1.2858395689592852, 1)
(3.3, 4.3, 4.3),(3.0, 2.0)(1.2201100350235683, 1)
(5.3, 3.3, 3.3),(2.0, 3.0)(1.8962782259357214, 1